In [3]:
from pyapacheatlas.auth import ServicePrincipalAuthentication
from pyapacheatlas.core import PurviewClient
from dotenv import load_dotenv
import os

load_dotenv()

tenant_id = os.environ.get("TENANT_ID") 
client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")
account_name = os.environ.get("PURVIEW_ACCOUNT")

auth = ServicePrincipalAuthentication(
    tenant_id = tenant_id, 
    client_id = client_id, 
    client_secret = client_secret
)

# Create a client to connect to your service.
client = PurviewClient(
    account_name = account_name,
    authentication = auth
)

In [4]:
client.get_all_typedefs()

{'enumDefs': [{'category': 'ENUM',
   'guid': '6e30d01e-f5d0-9799-08e7-09b4f276c09c',
   'createdBy': 'admin',
   'updatedBy': 'admin',
   'createTime': 1615787943686,
   'updateTime': 1615787943686,
   'version': 1,
   'name': 'glossary_term_status_value',
   'description': 'glossary_term_status_value',
   'typeVersion': '1.0',
   'lastModifiedTS': '1',
   'elementDefs': [{'value': 'Approved', 'ordinal': 0},
    {'value': 'Alert', 'ordinal': 1},
    {'value': 'Expired', 'ordinal': 2},
    {'value': 'Draft', 'ordinal': 3}]},
  {'category': 'ENUM',
   'guid': '42185ddb-29be-e911-ea52-890154eb07c0',
   'createdBy': 'admin',
   'updatedBy': 'admin',
   'createTime': 1615787943608,
   'updateTime': 1615787943973,
   'version': 2,
   'name': 'AtlasGlossaryTermRelationshipStatus',
   'description': 'TermRelationshipStatus defines how reliable the relationship is between two glossary terms',
   'typeVersion': '1.1',
   'serviceType': 'atlas_core',
   'lastModifiedTS': '2',
   'elementDefs': [